# Visualisation and Interactivity improvements
## Objectives
1. ~Complete write function that takes ODC xarray output and converts it to RGBA colormap whilst accounting for single bands~
2. ~Modify `mk_image_overlay` to also accept RGB(A) values~
3. Integrate reprojection to web mercator in parent function
4. Normalising dependent on input, no data, categories, bitmask, boolean
5. Sidecar / ipyleaflet auto update instead of creating new tabs
6. Transparency slider
7. ~Basemap plotting options~
8. Options for time slices
9. Display time of image on basemap

In [ ]:
import odc.algo
import odc.ui
from datacube import Datacube
import ipyleaflet
import datacube
import sys
import xarray as xr
import numpy as np
from typing import Tuple, Optional, List
from sidecar import Sidecar
from ipywidgets import IntSlider, widgets as w
from matplotlib import cm
import matplotlib.pyplot as plt
from IPython.display import Image, display
from matplotlib.colors import Normalize

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

sys.path.append('../../../Scripts')
from dea_datahandling import load_ard
from dea_plotting import rgb
from dea_plotting import display_map

In [ ]:
from matplotlib import cm
from matplotlib.colors import Normalize

def colorize(da, cmap='viridis', vmin=None, vmax=None):
    """
    Convert values in single band xarray to colormap values
    Can accept multiple time steps
    da - xarray.DataArray
    cmap - desired colormapping
    vmin / vmax - values for normalisation
    """
    normalized = Normalize(vmin=vmin, vmax=vmax)(da)
    colormapped = cm.get_cmap(cmap)(normalized, bytes=True)
    colormapped_xr = xr.DataArray(data=colormapped, 
                                  coords = da.coords, dims=da.dims+('band',))
    return colormapped_xr

In [ ]:
dc = Datacube(app='viz')

In [ ]:
# Create sidecar and map widget
sc = Sidecar(title='Map')

# Create map and add it to sidecar
m = ipyleaflet.Map(basemap=ipyleaflet.basemaps.Esri.WorldImagery, center=(-25, 133), zoom=3)

# Add Full Screen and Layers Controls
m.add_control(ipyleaflet.FullScreenControl())
m.add_control(ipyleaflet.LayersControl())

# Add the opactity slider
slider = w.FloatSlider(min=0, max=1, value=1,        # Opacity is valid in [0,1] range
                       orientation='vertical',       # Vertical slider is what we want
                       readout=False,                # No need to show exact value
                       layout=w.Layout(width='2em')) # Fine tune display layout: make it thinner
m.add_control(ipyleaflet.WidgetControl(widget=slider))

# Add map to sidecar
with sc:
    display(m)

In [ ]:
# Load data
query = {
    'x': (153.33, 153.425),
    'y': (-27.60, -27.665),
    'time': ('2018'),
    'output_crs': 'EPSG:3857',
    'resolution': (-30, 30),
    'group_by': 'solar_day'
}

ds = dc.load(product = 'ls8_nbart_geomedian_annual', **query)

print(ds)

Either run colorize and pass to `mk_image_overlay` or pass a multi band dataset to it.

In [ ]:
# Convert dataarray values to colormapping
cm_da = colorize(ds.red, cmap='Reds')

In [ ]:
#Add to map and center
image = odc.ui.mk_image_overlay(cm_da, layer_name='Image1')

In [ ]:
# Add layer to map
m.add_layer(image)
# Center map on new image
m.center = (query['y'][0], query['x'][0])

# Add the opacity slider to the new image
w.jslink((slider, 'value'),         
         (image, 'opacity') )